In [1]:
import os
import spacy
import pickle
import re
from nltk.stem import WordNetLemmatizer

In [2]:
lemmatizer = WordNetLemmatizer()

In [3]:
nlp_md = spacy.load('en_core_web_lg')
nlp_md.add_pipe('sentencizer')

In [4]:
filenames = [f for f in os.listdir("../data/large_data/articles_spacydocs") if ".pickle" in f]
filenames[:10]

['spacydoc_pop_id_28.pickle',
 'spacydoc_pop_id_199.pickle',
 'spacydoc_pop_id_253.pickle',
 'spacydoc_pop_id_502.pickle',
 'spacydoc_pop_id_55.pickle',
 'spacydoc_pop_id_353.pickle',
 'spacydoc_pop_id_402.pickle',
 'spacydoc_pop_id_36.pickle',
 'spacydoc_pop_id_230.pickle',
 'spacydoc_pop_id_187.pickle']

In [5]:
sourcepath = "../data/large_data/articles_spacydocs/"

In [6]:
filepath = sourcepath + "spacydoc_pop_id_17.pickle"

In [7]:
#with open(filepath, "rb") as f:
#    doc = pickle.load(f)
# test reading it back
doc = pickle.load(open("../data/large_data/doc_test.pickle", "rb"))

In [8]:
str(doc).count("right-wing")

2

In [9]:
[(i, sent) for i, sent in enumerate(doc.sents) if re.search("right-wing", str(sent))]

[(75,
  As a set of ideas, populism can be attached to different ideologies, ranging from left-wing to right-wing and from progressive to conservative.),
 (114,
  Various studies have indeed found that populist parties positioned at the fringes of the political spectrum tend to express Eurosceptic attitudes (Hooghe et al., 2002; De Vries and Edwards, 2009; Halikiopoulou et al., 2012), and that supporters of such parties (both left- and right-wing) tend to be rather Eurosceptic as well (Lubbers and Scheepers, 2007; Werts et al., 2013; Visser et al., 2014; Ramiro, 2016).)]

In [24]:
sent = list(doc.sents)[114]
sent

Various studies have indeed found that populist parties positioned at the fringes of the political spectrum tend to express Eurosceptic attitudes (Hooghe et al., 2002; De Vries and Edwards, 2009; Halikiopoulou et al., 2012), and that supporters of such parties (both left- and right-wing) tend to be rather Eurosceptic as well (Lubbers and Scheepers, 2007; Werts et al., 2013; Visser et al., 2014; Ramiro, 2016).

In [25]:
[(t.text, t.pos_, t.lemma_) for t in sent if t.is_stop==False]

[('studies', 'NOUN', 'study'),
 ('found', 'VERB', 'find'),
 ('populist', 'ADJ', 'populist'),
 ('parties', 'NOUN', 'party'),
 ('positioned', 'VERB', 'position'),
 ('fringes', 'NOUN', 'fringe'),
 ('political', 'ADJ', 'political'),
 ('spectrum', 'NOUN', 'spectrum'),
 ('tend', 'VERB', 'tend'),
 ('express', 'VERB', 'express'),
 ('Eurosceptic', 'ADJ', 'eurosceptic'),
 ('attitudes', 'NOUN', 'attitude'),
 ('(', 'PUNCT', '('),
 ('Hooghe', 'PROPN', 'Hooghe'),
 ('et', 'PROPN', 'et'),
 ('al', 'PROPN', 'al'),
 ('.', 'PROPN', '.'),
 (',', 'PUNCT', ','),
 ('2002', 'NUM', '2002'),
 (';', 'PUNCT', ';'),
 ('De', 'PROPN', 'De'),
 ('Vries', 'PROPN', 'Vries'),
 ('Edwards', 'PROPN', 'Edwards'),
 (',', 'PUNCT', ','),
 ('2009', 'NUM', '2009'),
 (';', 'PUNCT', ';'),
 ('Halikiopoulou', 'PROPN', 'Halikiopoulou'),
 ('et', 'PROPN', 'et'),
 ('al', 'PROPN', 'al'),
 ('.', 'PROPN', '.'),
 (',', 'PUNCT', ','),
 ('2012', 'NUM', '2012'),
 (')', 'PUNCT', ')'),
 (',', 'PUNCT', ','),
 ('supporters', 'NOUN', 'supporter'),
 (

In [26]:
print([t.text for t in sent])

['Various', 'studies', 'have', 'indeed', 'found', 'that', 'populist', 'parties', 'positioned', 'at', 'the', 'fringes', 'of', 'the', 'political', 'spectrum', 'tend', 'to', 'express', 'Eurosceptic', 'attitudes', '(', 'Hooghe', 'et', 'al', '.', ',', '2002', ';', 'De', 'Vries', 'and', 'Edwards', ',', '2009', ';', 'Halikiopoulou', 'et', 'al', '.', ',', '2012', ')', ',', 'and', 'that', 'supporters', 'of', 'such', 'parties', '(', 'both', 'left-', 'and', 'right-wing', ')', 'tend', 'to', 'be', 'rather', 'Eurosceptic', 'as', 'well', '(', 'Lubbers', 'and', 'Scheepers', ',', '2007', ';', 'Werts', 'et', 'al', '.', ',', '2013', ';', 'Visser', 'et', 'al', '.', ',', '2014', ';', 'Ramiro', ',', '2016', ')', '.']


In [27]:
lemmata_list = []
for t in sent:
    if t.is_stop==False:
        if (t.is_alpha==True):
            lemmata_list.append(t.lemma_)
        else:
            if "-" in t.text:
                start, hyphen, end = str(t.text).rpartition("-")
                new_lemma = "-".join([start, lemmatizer.lemmatize(end)])
                lemmata_list.append(new_lemma)
lemmata_list

['study',
 'find',
 'populist',
 'party',
 'position',
 'fringe',
 'political',
 'spectrum',
 'tend',
 'express',
 'eurosceptic',
 'attitude',
 'Hooghe',
 'et',
 'al',
 'De',
 'Vries',
 'Edwards',
 'Halikiopoulou',
 'et',
 'al',
 'supporter',
 'party',
 'left-',
 'right-wing',
 'tend',
 'eurosceptic',
 'Lubbers',
 'Scheepers',
 'Werts',
 'et',
 'al',
 'Visser',
 'et',
 'al',
 'Ramiro']

In [28]:
for ent in sent.ents:
    if ent.label_ == "PERSON":
        print(ent[-1].lemma_)

.
Vries
.
Lubbers
.


In [15]:
#ent = [ent for ent in sent.ents][0]

In [51]:
# postags = ["PROPN", "NOUN", "VERB", ]
def lemmata(spacyobject, filtered=True):
    if filtered==True:
        lemmata_list = []
        for t in spacyobject:
            if (t.is_stop==False) & (t.pos_ != "PROPN"):
                if (t.is_alpha==True):
                    lemmata_list.append(t.lemma_)
                else:
                    if "-" in t.text:
                        start, hyphen, end = str(t.text).rpartition("-")
                        new_lemma = "-".join([start, lemmatizer.lemmatize(end)]).lower()
                        lemmata_list.append(new_lemma)
        return  lemmata_list
    else:
        return  [t.lemma_ for t in spacyobject]
    # return [t.lemma_.lower() if t.pos=="PROPN" else t.lemma_ for t in spacyobject if (t.is_stop==False) & (t.is_alpha==True)]

In [52]:
[(t.text, t.pos_, t.lemma_) for t in sent]

[('Various', 'ADJ', 'various'),
 ('studies', 'NOUN', 'study'),
 ('have', 'AUX', 'have'),
 ('indeed', 'ADV', 'indeed'),
 ('found', 'VERB', 'find'),
 ('that', 'SCONJ', 'that'),
 ('populist', 'ADJ', 'populist'),
 ('parties', 'NOUN', 'party'),
 ('positioned', 'VERB', 'position'),
 ('at', 'ADP', 'at'),
 ('the', 'DET', 'the'),
 ('fringes', 'NOUN', 'fringe'),
 ('of', 'ADP', 'of'),
 ('the', 'DET', 'the'),
 ('political', 'ADJ', 'political'),
 ('spectrum', 'NOUN', 'spectrum'),
 ('tend', 'VERB', 'tend'),
 ('to', 'PART', 'to'),
 ('express', 'VERB', 'express'),
 ('Eurosceptic', 'ADJ', 'eurosceptic'),
 ('attitudes', 'NOUN', 'attitude'),
 ('(', 'PUNCT', '('),
 ('Hooghe', 'PROPN', 'Hooghe'),
 ('et', 'PROPN', 'et'),
 ('al', 'PROPN', 'al'),
 ('.', 'PROPN', '.'),
 (',', 'PUNCT', ','),
 ('2002', 'NUM', '2002'),
 (';', 'PUNCT', ';'),
 ('De', 'PROPN', 'De'),
 ('Vries', 'PROPN', 'Vries'),
 ('and', 'CCONJ', 'and'),
 ('Edwards', 'PROPN', 'Edwards'),
 (',', 'PUNCT', ','),
 ('2009', 'NUM', '2009'),
 (';', 'PUNCT

In [53]:
lemmata(sent)

['study',
 'find',
 'populist',
 'party',
 'position',
 'fringe',
 'political',
 'spectrum',
 'tend',
 'express',
 'eurosceptic',
 'attitude',
 'supporter',
 'party',
 'left-',
 'right-wing',
 'tend',
 'eurosceptic']

In [34]:
ents_inds = []
entstart_dict = {}
for ent in sent.ents:
    ents_inds.extend(range(ent.start + 1, ent.end))
    entstart_dict[ent.start] = "_".join(lemmata(ent, filtered=False))

In [35]:
entstart_dict

{3319: 'eurosceptic',
 3322: 'Hooghe_et_al_.',
 3327: '2002',
 3329: 'De_Vries',
 3332: 'Edwards',
 3334: '2009',
 3336: 'Halikiopoulou_et_al_.',
 3341: '2012',
 3360: 'eurosceptic',
 3364: 'Lubbers',
 3366: 'Scheepers',
 3368: '2007',
 3370: 'Werts_et_al_.',
 3375: '2013',
 3377: 'Visser_et_al_.',
 3382: '2014',
 3384: 'Ramiro',
 3386: '2016'}

In [36]:
ents_inds

[3323, 3324, 3325, 3330, 3337, 3338, 3339, 3371, 3372, 3373, 3378, 3379, 3380]

In [37]:
sent

Various studies have indeed found that populist parties positioned at the fringes of the political spectrum tend to express Eurosceptic attitudes (Hooghe et al., 2002; De Vries and Edwards, 2009; Halikiopoulou et al., 2012), and that supporters of such parties (both left- and right-wing) tend to be rather Eurosceptic as well (Lubbers and Scheepers, 2007; Werts et al., 2013; Visser et al., 2014; Ramiro, 2016).

In [38]:
def get_sentence_lemmata(sent):
    ents_inds = []
    entstart_dict = {}
    for ent in sent.ents:
        ents_inds.extend(range(ent.start + 1, ent.end))
        if ent.label_ == "PERSON":
            entstart_dict[ent.start] = ent[-1].lemma_
        else:
            entstart_dict[ent.start] = "_".join(lemmata(ent, filtered=False))
    sentence_lemmata = []
    for i in range(sent.start, sent.end):
        if i in entstart_dict.keys():
            sentence_lemmata.append(entstart_dict[i])
        else:
            if i not in ents_inds:
                t = sent[i - sent.start]
                if t.is_stop==False:
                    if (t.is_alpha==True):
                        sentence_lemmata.append(t.lemma_)
                    else:
                        if "-" in t.text:
                            start, hyphen, end = str(t.text).rpartition("-")
                            new_lemma = "-".join([start, lemmatizer.lemmatize(end)])
                            sentence_lemmata.append(new_lemma)
    return sentence_lemmata

In [39]:
sent

Various studies have indeed found that populist parties positioned at the fringes of the political spectrum tend to express Eurosceptic attitudes (Hooghe et al., 2002; De Vries and Edwards, 2009; Halikiopoulou et al., 2012), and that supporters of such parties (both left- and right-wing) tend to be rather Eurosceptic as well (Lubbers and Scheepers, 2007; Werts et al., 2013; Visser et al., 2014; Ramiro, 2016).

In [40]:
get_sentence_lemmata(sent)

['study',
 'find',
 'populist',
 'party',
 'position',
 'fringe',
 'political',
 'spectrum',
 'tend',
 'express',
 'eurosceptic',
 'attitude',
 '.',
 '2002',
 'Vries',
 'Edwards',
 '2009',
 '.',
 '2012',
 'supporter',
 'party',
 'left-',
 'right-wing',
 'tend',
 'eurosceptic',
 'Lubbers',
 'Scheepers',
 '2007',
 'Werts_et_al_.',
 '2013',
 '.',
 '2014',
 'Ramiro',
 '2016']

In [49]:
lemmatized_sents = [" ".join(get_sentence_lemmata(sent)) for sent in doc.sents]
lemmatized_sents[:5]

['abstract five_decade term consociationalism debut comparative politic divided society huge body international literature spark continue grow',
 'era populism polarization interest facet consociationalism all-time high',
 'introduction special issue half_a_Century consociationalism case Comparisons give impression contemporary state consociational research agenda',
 'addition introduction suggest potentially rewarding new avenue leadership study caution conﬂation consociationalism power sharing',
 'voice skepticism suggest promise large n-quantitative study ﬁeld dominate case-based research']

In [139]:
with open("testfile.txt", "w") as f:
    f.writelines("\n".join(lemmatized_sents))

In [48]:
lemmatized_sents = [" ".join(lemmata(sent)) for sent in doc.sents]
lemmatized_sents[:5]

['abstract decade term consociationalism debut comparative politic divided society huge body international literature spark continue grow',
 'era populism polarization interest facet consociationalism all-time high',
 'introduction special issue half consociationalism case give impression contemporary state consociational research agenda',
 'addition introduction suggest potentially rewarding new avenue leadership study caution conﬂation consociationalism power sharing',
 'voice skepticism suggest promise large N-quantitative study ﬁeld dominate case-based research']

# Lemmata with named entities

In [142]:
!mkdir ../data/large_data/articles_lemmata

mkdir: ../data/large_data/articles_lemmata: File exists


In [143]:
filenames[0]

'spacydoc_pop_id_28.pickle'

In [25]:
sourcepath = "../data/large_data/articles_spacydocs/"
destpath = "../data/large_data/articles_lemmata/"
for filename in filenames:
    filepath = sourcepath + filename
    with open(filepath, "rb") as f:
        doc = pickle.load(f)
    lemmatized_sents = [" ".join(get_sentence_lemmata(sent)) for sent in doc.sents]
    with open(destpath + filename.replace("spacydoc", "lemmata").replace(".pickle", ".txt"), "w") as f:
        f.writelines("\n".join(lemmatized_sents))


# Lemmata without named entities

In [43]:
!mkdir ../data/large_data/articles_lemmata_min

In [54]:
sourcepath = "../data/large_data/articles_spacydocs/"
destpath = "../data/large_data/articles_lemmata_min/"
for filename in filenames:
    filepath = sourcepath + filename
    with open(filepath, "rb") as f:
        doc = pickle.load(f)
    lemmatized_sents = [" ".join(lemmata(sent)) for sent in doc.sents]
    with open(destpath + filename.replace("spacydoc", "lemmata").replace(".pickle", ".txt"), "w") as f:
        f.writelines("\n".join(lemmatized_sents))